<a href="https://colab.research.google.com/github/Art-phys/Lesson_HF_LR/blob/main/Lesson_HF_RL_Unit4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Блок 4: Создайте свой первый алгоритм обучения с глубоким подкреплением с помощью PyTorch: Reinforce. И проверьте его надежность 💪

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit6/thumbnail.png" alt="thumbnail" width="50%"/>

В этом блокноте вы с нуля создадите свой первый алгоритм обучения с глубоким подкреплением: Reinforcement (также называемый градиентом политики Монте-Карло).

Усиление (Reinforce) - это *метод, основанный на политике*: алгоритм глубокого обучения с подкреплением, который пытается **оптимизировать политику напрямую, не используя функцию значения действия**.

Точнее, Усиление (Reinforce) - это *Метод градиента политики*, подкласс *методов, основанных на политике*, который направлен **на оптимизацию политики непосредственно путем оценки весов оптимальной политики с использованием градиентного подъема**.

Чтобы проверить его надежность, мы собираемся обучить его в двух разных простых средах:
- Cartpole-v1 (Столб тележки-v1)
- PixelcopterEnv (Пиксельный вертолет)

⬇️ Вот пример того, чего **вы достигнете в конце этом блокноте.** ⬇️

  <img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit6/envs.gif" alt="Environments" width="50%"/>


### 🎮 Окружающая среда: 

- [CartPole-v1](https://www.gymlibrary.dev/environments/classic_control/cart_pole/)
- [PixelCopter](https://pygame-learning-environment.readthedocs.io/en/latest/user/games/pixelcopter.html)

### 📚 RL-Библиотека: 

- Python
- PyTorch


 [GitHub Repo](https://github.com/huggingface/deep-rl-class/issues).

## Цели этого блокнота 🏆
В конце вы будете:
- Уметь **с нуля закодировать алгоритм усиления с использованием PyTorch.**
- Иметь возможность **протестировать надежность вашего агента, используя простые среды.**
- Иметь возможность **отправить своего обученного агента в центр** с хорошим видеоповтором и оценочным баллом 🔥 .

## Эта тетрадь взята из курса обучения с глубоким подкреплением
<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/deep-rl-course-illustration.jpg" alt="Deep RL Course illustration" width="50%"/>

На этом бесплатном курсе вы:

- 📖 Изучите глубокое обучение с подкреплением в **теории и на практике**.
- 🧑 💻 Научитесь **использовать известные библиотеки Deep RL**, такие как Stable Baselines3, RL Baselines3 Zoo, CleanRL и Sample Factory 2.0.
- 🤖 Обучите **агентов в уникальных условиях**

## Предпосылки 🏗️
Прежде чем погрузиться в блокнот, вам нужно:

🔲 📚 [Изучите градиенты политики, прочитав Unit 4](https://huggingface.co/deep-rl-course/unit4/introduction)

# Давайте запрограммируем алгоритм усиления с нуля 🔥


Чтобы пройти этот практический подход для сертификации, вам необходимо отправить свои обученные модели в Центр.

- Получите результат >= 350 для `Cart pole-v1`.
- Получите результат >= 5 для `Pixel Copter`.

Чтобы узнать свой результат, перейдите в таблицу лидеров и найдите свою модель, **the result = mean_reward - std_reward**. **Если вы не видите свою модель в таблице лидеров, перейдите в нижнюю часть страницы таблицы лидеров и нажмите на кнопку обновить**.

Для получения дополнительной информации о процессе сертификации ознакомьтесь с этим разделом 👉 https://huggingface.co/deep-rl-course/en/unit0/introduction#certification-process

# Совет 💡
Лучше запустить этот colab в копии на вашем Google Диске, чтобы **в случае истечения времени ожидания** у вас все еще была сохраненная записная книжка на вашем Google Диске, и вам не нужно было заполнять все с нуля.

Для этого вы можете либо нажать `Ctrl + S` либо `File > Save a copy in Google Drive.`

## Запустить на GPU 💪
- Чтобы **ускорить обучение агента**, мы будем использовать графический процессор GPU. Перейдите в `Runtime > Change Runtime type`

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/gpu-step1.jpg" alt="GPU Step 1" width="50%">

- `Hardware Accelerator > GPU`

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/gpu-step2.jpg" alt="GPU Step 2" width="50%">

## Создание виртуального дисплея #

Во время записи нам нужно будет сгенерировать видео-повтор. Для этого с помощью colab **нам нужен виртуальный экран, чтобы иметь возможность отображать среду** (и, таким образом, записывать кадры).

Следовательно, в следующей ячейке будут установлены библиотеки, а также создан и запущен виртуальный экран 🖥

In [1]:
%%capture
!apt install python-opengl
!apt install ffmpeg
!apt install xvfb
!pip install pyvirtualdisplay
!pip install pyglet==1.5.1

In [2]:
# Virtual display
from pyvirtualdisplay import Display

virtual_display = Display(visible=0, size=(1400, 900))
virtual_display.start()

## Установка зависимостей #
Первым шагом является установка зависимостей:

- `gym`
- `gym-games`: Дополнительные условия в тренажерном зале, созданные с помощью Pygame.
- `huggingface_hub`: 🤗 работает как центральное место, где любой желающий может делиться моделями и наборами данных и изучать их. В нем есть управление версиями, метрики, визуализации и другие функции, которые позволят вам легко сотрудничать с другими.

Вы можете увидеть здесь все доступные модели усилителей 👉 https://huggingface.co/models?other=reinforce

И вы можете найти все модели обучения с глубоким подкреплением здесь 👉 https://huggingface.co/models?pipeline_tag=reinforcement-learning

In [3]:
!pip install -r https://raw.githubusercontent.com/huggingface/deep-rl-class/main/notebooks/unit4/requirements-unit4.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/ntasfi/PyGame-Learning-Environment.git to /tmp/pip-req-build-qyqgvrtw
  Running command git clone --filter=blob:none --quiet https://github.com/ntasfi/PyGame-Learning-Environment.git /tmp/pip-req-build-qyqgvrtw
  Resolved https://github.com/ntasfi/PyGame-Learning-Environment.git to commit 3dbe79dc0c35559bb441b9359948aabf9bb3d331
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/qlan3/gym-games.git to /tmp/pip-req-build-v0kseeo1
  Running command git clone --filter=blob:none --quiet https://github.com/qlan3/gym-games.git /tmp/pip-req-build-v0kseeo1
  Resolved https://github.com/qlan3/gym-games.git to commit fe4cc2d4e3de06c7e6593a10e4b01d3b1c5a99d6
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.9/26.9 MB 46.5 MB/s eta 0:

## Импортируйте пакеты 📦
В дополнение к импорту установленных библиотек, мы также импортируем:

- `imageio`: Библиотека, которая поможет нам сгенерировать повтор видео


In [4]:
import numpy as np

from collections import deque

import matplotlib.pyplot as plt
%matplotlib inline

# PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical

# Gym
import gym
import gym_pygame

# Hugging Face Hub
from huggingface_hub import notebook_login # Войти в учетную запись Hugging Face, чтобы иметь возможность загружать модели в Хаб.
import imageio

## Проверьте, есть ли у нас графический процессор

Если у вас есть GPU, вы должны увидеть `device:cuda0`

In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [6]:
print(device)

cpu


Теперь мы готовы реализовать наш алгоритм усиления 🔥

# Первый агент: Играем в Картпол-v1 🤖

## Создайте среду CartPole и поймите, как это работает
### [Окружающая среда 🎮 ](https://www.gymlibrary.dev/environments/classic_control/cart_pole/)


### Почему мы используем такую простую среду, как CartPole-v1?
Как описано в [Советах и рекомендациях по обучению с подкреплением](https://stable-baselines3.readthedocs.io/en/master/guide/rl_tips.html),когда вы внедряете свой агент с нуля, вам нужно **убедиться, что он работает правильно, и найти ошибки в простых средах, прежде чем углубляться**. Поскольку находить ошибки будет намного проще в простых средах.


> Попробуйте подать какой-нибудь “признак жизни” в отношении проблем с игрушками


> Проверьте реализацию, заставив ее запускаться во все более сложных средах (вы можете сравнить результаты с RL zoo). Обычно для этого шага необходимо выполнить оптимизацию по гиперпараметру.
___
### Среда CartPole-v1

> Шест прикреплен с помощью шарнира без привода к тележке, которая движется по рельсам без трения. Маятник устанавливается вертикально на тележку, и цель состоит в том, чтобы уравновесить шест, прикладывая силы в левом и правом направлениях к тележке.



Итак, мы начнем с CartPole-v1. Цель состоит в том, чтобы толкать тележку влево или вправо **так, чтобы шест оставался в равновесии.**

Эпизод заканчивается, если:
- Угол наклона шеста больше, чем ±12°
- Положение тележки больше, чем ±2.4
- Продолжительность эпизода больше, чем 500

Мы получаем вознаграждение 💰 в размере +1 каждый раз, когда полюс остается в равновесии.

In [7]:
env_id = "CartPole-v1"
# Создание env
env = gym.make(env_id)

# Создание среды оценки
eval_env = gym.make(env_id)

# Получение пространства состояний и пространства действий
s_size = env.observation_space.shape[0]
a_size = env.action_space.n

/usr/local/lib/python3.8/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.8/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [12]:
print("_____OBSERVATION SPACE_____ \n")
print("The State Space is: ", s_size)
print("Sample observation", env.observation_space.sample()) # Получение случайного наблюдения

_____OBSERVATION SPACE_____ 

The State Space is:  4
Sample observation [ 8.8104260e-01 -2.8916262e+38 -2.6740712e-01 -3.1581152e+38]


In [13]:
print("\n _____ACTION SPACE_____ \n")
print("The Action Space is: ", a_size)
print("Action Space Sample", env.action_space.sample()) # Получение случайного действия


 _____ACTION SPACE_____ 

The Action Space is:  2
Action Space Sample 1


## Давайте создадим Reinforce архитектуру
Эта реализация основана на двух реализациях:
- [Официальный пример обучения с подкреплением PyTorch](https://github.com/pytorch/examples/blob/main/reinforcement_learning/reinforce.py)
- [Udacity Reinforce](https://github.com/udacity/deep-reinforcement-learning/blob/master/reinforce/REINFORCE.ipynb)
- [Улучшение интеграции с помощью Chris1nexus](https://github.com/huggingface/deep-rl-class/pull/95)

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit6/reinforce.png" alt="Reinforce" width="50%"/>

Итак, мы хотим:
- Два полностью соединенных слоя (fc1 и fc2).
- Использование ReLU в качестве функции активации fc1
- Использование Softmax для вывода распределения вероятностей по действиям

In [8]:
class Policy(nn.Module):
    def __init__(self, s_size, a_size, h_size):
        super(Policy, self).__init__()
        # Создайте два полностью соединенных (fc-fully connected ) слоя

    def forward(self, x):
        # Определите прямой проход
        # состояние переходит в fc1, затем мы применяем функцию активации ReLU
        # выходы fc1 передаются в fc2
        # Мы выводим softmax
    
    def act(self, state):
        """
        Учитывая состояние, определите действие
        """
        state = torch.from_numpy(state).float().unsqueeze(0).to(device)
        probs = self.forward(state).cpu()
        m = Categorical(probs)
        action = np.argmax(m)
        return action.item(), m.log_prob(action)

IndentationError: ignored

### Solution

In [9]:
class Policy(nn.Module):
    def __init__(self, s_size, a_size, h_size):
        super(Policy, self).__init__()
        self.fc1 = nn.Linear(s_size, h_size)
        self.fc2 = nn.Linear(h_size, a_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.softmax(x, dim=1)
    
    def act(self, state):
        state = torch.from_numpy(state).float().unsqueeze(0).to(device)
        probs = self.forward(state).cpu()
        m = Categorical(probs)
        action = np.argmax(m)
        return action.item(), m.log_prob(action)

Я совершаю ошибку, можете ли вы догадаться, где?

- Чтобы выяснить это, давайте сделаем пас вперед:

In [10]:
debug_policy = Policy(s_size, a_size, 64).to(device)
debug_policy.act(env.reset())

ValueError: ignored

- Здесь мы видим, что ошибка гласит `ValueError: аргумент value для log_prob должен быть тензором`

- Это означает, что `action` в `m.log_prob(action)` должно быть, тензором **, но это не так.**

- Ты знаешь почему? Проверьте функцию act и попытайтесь понять, почему она не работает.

Совет 💡 : Что-то не так в этой реализации. Помните, что **мы хотим выбрать действие из распределения вероятностей по действиям**.


### Правильное решение

In [11]:
class Policy(nn.Module):
    def __init__(self, s_size, a_size, h_size):
        super(Policy, self).__init__()
        self.fc1 = nn.Linear(s_size, h_size)
        self.fc2 = nn.Linear(h_size, a_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.softmax(x, dim=1)
    
    def act(self, state):
        state = torch.from_numpy(state).float().unsqueeze(0).to(device)
        probs = self.forward(state).cpu()
        m = Categorical(probs)
        action = m.sample()
        return action.item(), m.log_prob(action)

Используя CartPole, отлаживать было проще, поскольку **мы знаем, что ошибка возникает из-за нашей интеграции, а не из-за нашей простой среды**.

- Поскольку **мы хотим выбрать действие из распределения вероятностей по действиям**, мы не можем использовать `action = np.argmax(m)`, поскольку оно всегда будет выводить действие с наибольшей вероятностью.

- Нам нужно заменить на `action = m.sample()`, который будет отбирать действие из распределения вероятностей P(.|s)

### Давайте построим алгоритм усиленного обучения
Это псевдокод алгоритма усиления:

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit6/pg_pseudocode.png" alt="Policy gradient pseudocode"  width="70%"/>

- Когда мы вычисляем доходность Gt (строка 6), мы видим, что мы вычисляем сумму дисконтированных вознаграждений, **начиная с временного интервала t**.

- почему? Поскольку наша политика должна только **подкреплять действия на основе последствий**: таким образом, награды, полученные до совершения действия, бесполезны (поскольку они были получены не из-за действия), **важны только те, которые приходят после действия**.

- Прежде чем кодировать это, вы должны прочитать этот раздел [ "не позволяйте прошлому отвлекать вас"](https://spinningup.openai.com/en/latest/spinningup/rl_intro3.html#don-t-let-the-past-distract-you) это объясняет, почему мы используем градиент политики "вознаграждение за переход".

Мы используем интересную технику, закодированную [Chris1nexus](https://github.com/Chris1nexus) чтобы **эффективно вычислять доходность на каждом временном шаге**. В комментариях объяснялась процедура. Не стесняйтесь также [проверить объяснение PR](https://github.com/huggingface/deep-rl-class/pull/95)
Но в целом идея состоит в том, чтобы **эффективно вычислять доходность на каждом временном шаге**.

Второй вопрос, который вы можете задать, звучит так: **почему мы минимизируем потери**? Вы говорили о градиентном подъеме, а не о градиентном спуске?

- Мы хотим максимизировать нашу функцию полезности $J(\theta)$ но в PyTorch, как и в Tensorflow, лучше **минимизировать целевую функцию.**
    - Итак, допустим, мы хотим усилить действие 3 на определенном временном шаге. Перед тренировкой этого действия P равно 0.25.
    - Итак, мы хотим изменить $\theta$ таким образом, чтобы $\pi_\theta(a_3|s; \theta) > 0.25$
    - Поскольку все P должны суммироваться до 1, то функция max $\pi_\theta(a_3|s; \theta)$ **сведет к минимуму вероятность других действий.**
    - Таким образом мы должны в PyTorch **, использовать функцию min $(1 - \pi_\theta(a_3|s; \theta)$).**
    - Эта функция потерь приближается к 0 по мере того, как $\pi_\theta(a_3|s; \theta)$  приближается к 1.
    - Таким образом градиент стремится max $\pi_\theta(a_3|s; \theta)$


In [17]:
def reinforce(policy, optimizer, n_training_episodes, max_t, gamma, print_every):
    # Помогите нам подсчитать балл во время тренировки
    scores_deque = deque(maxlen=100)
    scores = []
    # Строка 3 псевдокода
    for i_episode in range(1, n_training_episodes+1):
        saved_log_probs = []
        rewards = []
        state = # ЗАДАЧА: перезагрузите среду
        # Строка 4 псевдокода
        for t in range(max_t):
            action, log_prob = # ЗАДАЧА: получить действие
            saved_log_probs.append(log_prob)
            state, reward, done, _ = # ЗАДАЧА: сделайте шаг в среде env
            rewards.append(reward)
            if done:
                break 
        scores_deque.append(sum(rewards))
        scores.append(sum(rewards))
        
        # Строка 6 псевдокода: вычисление возвращаемой награды
        returns = deque(maxlen=max_t) 
        n_steps = len(rewards) 
        # Вычислите дисконтированную доходность на каждом временном шаге
        # как сумму доходности с гамма-дисконтированной доходностью в момент t(G_t) + вознаграждение в момент t

        # За O(N) время, где N - количество временных шагов
        # (это определение дисконтированного дохода G_t следует определению этого количества
        # показано на странице 44 2-го черновика Sutton&Barto 2017)
        # G_t = r_(t+1) + r_(t+2) + ...

        # Учитывая эту формулировку, возвраты на каждом временном шаге t могут быть вычислены
        # путем повторного использования вычисленных будущих возвратов G_(t + 1) для вычисления текущего возврата G_t
        # G_t = r_(t+1) + гамма*G_(t+1)
        # G_(t-1) = r_t + гамма * G_t
        # (это соответствует подходу динамического программирования, с помощью которого мы запоминаем решения, чтобы
        # избежать их многократного вычисления)

        # Это правильно, поскольку приведенное выше эквивалентно (см. также стр. 46 2-го черновика Sutton&Barto 2017)
        # G_(t-1) = r_t + гамма*r_(t+1) + гамма*гамма*r_(t+2) + ...


        ## Учитывая вышесказанное, мы вычисляем доходность на временном шаге t как:
        # гамма[t] * возврат[t] + награда[t]
        #
        ## Мы вычисляем это, начиная с последнего временного шага и заканчивая первым, по порядку
        ## использовать представленную выше формулу и избежать избыточных вычислений, которые потребовались
        ## бы если бы мы делали это от начала до конца.

        ## Следовательно, очередь "returns" будет содержать возвраты в хронологическом порядке, от t=0 до t=n_steps
        ## благодаря функции appendleft(), которая позволяет добавлять к позиции 0 за постоянное время O(1)
        ## обычный список python вместо этого потребовал бы O (N) для этого.
        for t in range(n_steps)[::-1]:
            disc_return_t = (returns[0] if len(returns)>0 else 0)
            returns.appendleft(    ) # ЗАДАЧА: завершите здесь       
       
        ## стандартизация результатов используется для того, чтобы сделать обучение более стабильным
        eps = np.finfo(np.float32).eps.item()
        
        ## eps - это наименьшее представимое значение с плавающей точкой, которое
        # добавляется к стандартному отклонению доходности, чтобы избежать числовой нестабильности
        returns = torch.tensor(returns)
        returns = (returns - returns.mean()) / (returns.std() + eps)
        
        # Строка 7:
        policy_loss = []
        for log_prob, disc_return in zip(saved_log_probs, returns):
            policy_loss.append(-log_prob * disc_return)
        policy_loss = torch.cat(policy_loss).sum()
        
        # Строка 8: PyTorch предпочитает градиентный спуск 
        optimizer.zero_grad()
        policy_loss.backward()
        optimizer.step()
        
        if i_episode % print_every == 0:
            print('Episode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_deque)))
        
    return scores

SyntaxError: ignored

#### Решение

In [13]:
def reinforce(policy, optimizer, n_training_episodes, max_t, gamma, print_every):
    # Help us to calculate the score during the training
    scores_deque = deque(maxlen=100)
    scores = []
    # Line 3 of pseudocode
    for i_episode in range(1, n_training_episodes+1):
        saved_log_probs = []
        rewards = []
        state = env.reset()
        # Line 4 of pseudocode
        for t in range(max_t):
            action, log_prob = policy.act(state)
            saved_log_probs.append(log_prob)
            state, reward, done, _ = env.step(action)
            rewards.append(reward)
            if done:
                break 
        scores_deque.append(sum(rewards))
        scores.append(sum(rewards))
        
        # Line 6 of pseudocode: calculate the return
        returns = deque(maxlen=max_t) 
        n_steps = len(rewards) 
        # Compute the discounted returns at each timestep,
        # as 
        #      the sum of the gamma-discounted return at time t (G_t) + the reward at time t
        #
        # In O(N) time, where N is the number of time steps
        # (this definition of the discounted return G_t follows the definition of this quantity 
        # shown at page 44 of Sutton&Barto 2017 2nd draft)
        # G_t = r_(t+1) + r_(t+2) + ...
        
        # Given this formulation, the returns at each timestep t can be computed 
        # by re-using the computed future returns G_(t+1) to compute the current return G_t
        # G_t = r_(t+1) + gamma*G_(t+1)
        # G_(t-1) = r_t + gamma* G_t
        # (this follows a dynamic programming approach, with which we memorize solutions in order 
        # to avoid computing them multiple times)
        
        # This is correct since the above is equivalent to (see also page 46 of Sutton&Barto 2017 2nd draft)
        # G_(t-1) = r_t + gamma*r_(t+1) + gamma*gamma*r_(t+2) + ...
        
        
        ## Given the above, we calculate the returns at timestep t as: 
        #               gamma[t] * return[t] + reward[t]
        #
        ## We compute this starting from the last timestep to the first, in order
        ## to employ the formula presented above and avoid redundant computations that would be needed 
        ## if we were to do it from first to last.
        
        ## Hence, the queue "returns" will hold the returns in chronological order, from t=0 to t=n_steps
        ## thanks to the appendleft() function which allows to append to the position 0 in constant time O(1)
        ## a normal python list would instead require O(N) to do this.
        for t in range(n_steps)[::-1]:
            disc_return_t = (returns[0] if len(returns)>0 else 0)
            returns.appendleft( gamma*disc_return_t + rewards[t]   )    
            
        ## standardization of the returns is employed to make training more stable
        eps = np.finfo(np.float32).eps.item()
        ## eps is the smallest representable float, which is 
        # added to the standard deviation of the returns to avoid numerical instabilities        
        returns = torch.tensor(returns)
        returns = (returns - returns.mean()) / (returns.std() + eps)
        
        # Line 7:
        policy_loss = []
        for log_prob, disc_return in zip(saved_log_probs, returns):
            policy_loss.append(-log_prob * disc_return)
        policy_loss = torch.cat(policy_loss).sum()
        
        # Line 8: PyTorch prefers gradient descent 
        optimizer.zero_grad()
        policy_loss.backward()
        optimizer.step()
        
        if i_episode % print_every == 0:
            print('Episode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_deque)))
        
    return scores

## Тренируй это
- Теперь мы готовы обучать нашего агента.
- Но сначала мы определяем переменную, содержащую все обучающие гиперпараметры.
- Вы можете изменить параметры тренировки (и должны 😉)

In [29]:
cartpole_hyperparameters = {
    "h_size": 64,
    "n_training_episodes": 1000,
    "n_evaluation_episodes": 10,
    "max_t": 1000,
    "gamma": 1.0,
    "lr": 1e-2,
    "env_id": env_id,
    "state_space": s_size,
    "action_space": a_size,
}

In [30]:
# Создайте политику и разместите ее на устройстве
cartpole_policy = Policy(cartpole_hyperparameters["state_space"], cartpole_hyperparameters["action_space"], cartpole_hyperparameters["h_size"]).to(device)
cartpole_optimizer = optim.Adam(cartpole_policy.parameters(), lr=cartpole_hyperparameters["lr"])

In [31]:
scores = reinforce(cartpole_policy,
                   cartpole_optimizer,
                   cartpole_hyperparameters["n_training_episodes"], 
                   cartpole_hyperparameters["max_t"],
                   cartpole_hyperparameters["gamma"], 
                   100)

Episode 100	Average Score: 164.31
Episode 200	Average Score: 247.84
Episode 300	Average Score: 372.64
Episode 400	Average Score: 381.96
Episode 500	Average Score: 50.25
Episode 600	Average Score: 257.81
Episode 700	Average Score: 500.00
Episode 800	Average Score: 398.98
Episode 900	Average Score: 450.93
Episode 1000	Average Score: 485.24


## Определите метод оценки 📝
- Здесь мы определяем метод оценки, который мы собираемся использовать для тестирования нашего Reinforce агента.

In [25]:
def evaluate_agent(env, max_steps, n_eval_episodes, policy):
  """
  Оцените агента на количестве эпизодов `n_eval_episodes` и верните среднее вознаграждение и std вознаграждение.
  :param env: Среда оценки
  :param n_eval_episodes: Количество эпизодов для оценки агента
  :param policy: Reinforce агент
  """
  episode_rewards = []
  for episode in range(n_eval_episodes):
    state = env.reset()
    step = 0
    done = False
    total_rewards_ep = 0
    
    for step in range(max_steps):
      action, _ = policy.act(state)
      new_state, reward, done, info = env.step(action)
      total_rewards_ep += reward
        
      if done:
        break
      state = new_state
    episode_rewards.append(total_rewards_ep)
  mean_reward = np.mean(episode_rewards)
  std_reward = np.std(episode_rewards)

  return mean_reward, std_reward

## Оцените нашего агента 📈

In [24]:
evaluate_agent(eval_env, 
               cartpole_hyperparameters["max_t"], 
               cartpole_hyperparameters["n_evaluation_episodes"],
               cartpole_policy)

NameError: ignored

### Опубликуйте нашу обученную модель на хабе 🔥
Теперь, когда мы увидели, что получили хорошие результаты после обучения, мы можем опубликовать нашу обученную модель на хабе 🤗 с помощью одной строки кода.

Вот пример карточки модели:

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit6/modelcard.png"/>

### Нажмите на концентратор
#### Не изменяйте этот код

In [14]:
from huggingface_hub import HfApi, snapshot_download
from huggingface_hub.repocard import metadata_eval_result, metadata_save

from pathlib import Path
import datetime
import json
import imageio

import tempfile

import os

In [15]:
def record_video(env, policy, out_directory, fps=30):
  """
  Generate a replay video of the agent
  :param env
  :param Qtable: Qtable of our agent
  :param out_directory
  :param fps: how many frame per seconds (with taxi-v3 and frozenlake-v1 we use 1)
  """
  images = []  
  done = False
  state = env.reset()
  img = env.render(mode='rgb_array')
  images.append(img)
  while not done:
    # Take the action (index) that have the maximum expected future reward given that state
    action, _ = policy.act(state)
    state, reward, done, info = env.step(action) # We directly put next_state = state for recording logic
    img = env.render(mode='rgb_array')
    images.append(img)
  imageio.mimsave(out_directory, [np.array(img) for i, img in enumerate(images)], fps=fps)

In [16]:
def push_to_hub(repo_id, 
                model,
                hyperparameters,
                eval_env,
                video_fps=30
                ):
  """

    Оцените, сгенерируйте видео и загрузите модель в Hugging Face Hub.
  Этот метод выполняет полный конвейер:
  - Он оценивает модель
  - Он генерирует карточку модели
  - Он генерирует повторное видео агента
  - Это подталкивает все к центру

  :param repo_id: идентификатор хранилища моделей из центра Hugging Face
  :param model: модель pytorch, которую мы хотим сохранить
  :param hyperparameters: обучающие гиперпараметры
  :param eval_env: среда оценки
  :param video_fps: сколько кадров в секунду требуется для записи нашего видеоповтора
  """

  _, repo_name = repo_id.split("/")
  api = HfApi()
  
  # Step 1: Создайте репозиторий
  repo_url = api.create_repo(
        repo_id=repo_id,
        exist_ok=True,
  )

  with tempfile.TemporaryDirectory() as tmpdirname:
    local_directory = Path(tmpdirname)
  
    # Step 2: Сохраните модель
    torch.save(model, local_directory / "model.pt")

    # Step 3: Сохраните гиперпараметры в JSON
    with open(local_directory / "hyperparameters.json", "w") as outfile:
      json.dump(hyperparameters, outfile)
    
    # Step 4: Оцените модель и создайте JSON
    mean_reward, std_reward = evaluate_agent(eval_env, 
                                            hyperparameters["max_t"],
                                            hyperparameters["n_evaluation_episodes"], 
                                            model)
    # Получить дату и время
    eval_datetime = datetime.datetime.now()
    eval_form_datetime = eval_datetime.isoformat()

    evaluate_data = {
          "env_id": hyperparameters["env_id"], 
          "mean_reward": mean_reward,
          "n_evaluation_episodes": hyperparameters["n_evaluation_episodes"],
          "eval_datetime": eval_form_datetime,
    }

    # Сохраните файл JSON
    with open(local_directory / "results.json", "w") as outfile:
        json.dump(evaluate_data, outfile)

    # Step 5: Создайте карточку модели
    env_name = hyperparameters["env_id"]
    
    metadata = {}
    metadata["tags"] = [
          env_name,
          "reinforce",
          "reinforcement-learning",
          "custom-implementation",
          "deep-rl-class"
      ]

    # Добавление показателей
    eval = metadata_eval_result(
        model_pretty_name=repo_name,
        task_pretty_name="reinforcement-learning",
        task_id="reinforcement-learning",
        metrics_pretty_name="mean_reward",
        metrics_id="mean_reward",
        metrics_value=f"{mean_reward:.2f} +/- {std_reward:.2f}",
        dataset_pretty_name=env_name,
        dataset_id=env_name,
      )

    # Объединяет оба словаря
    metadata = {**metadata, **eval}

    model_card = f"""
  # **Reinforce** Agent playing **{env_id}**
  This is a trained model of a **Reinforce** agent playing **{env_id}** .
  To learn to use this model and train yours check Unit 4 of the Deep Reinforcement Learning Course: https://huggingface.co/deep-rl-course/unit4/introduction
  """

    readme_path = local_directory / "README.md"
    readme = ""
    if readme_path.exists():
        with readme_path.open("r", encoding="utf8") as f:
          readme = f.read()
    else:
      readme = model_card

    with readme_path.open("w", encoding="utf-8") as f:
      f.write(readme)

    # Save our metrics to Readme metadata
    metadata_save(readme_path, metadata)

    # Step 6: Record a video
    video_path =  local_directory / "replay.mp4"
    record_video(env, model, video_path, video_fps)

    # Step 7. Push everything to the Hub
    api.upload_folder(
          repo_id=repo_id,
          folder_path=local_directory,
          path_in_repo=".",
    )

    print(f"Ваша модель помещена в центр. Вы можете просмотреть свою модель здесь: {repo_url}")

### .

Используя `push_to_hub` **, вы оцениваете, записываете повтор, генерируете карточку модели вашего агента и отправляете ее в Хаб**.

Таким образом:
- Вы можете **продемонстрировать нашу работу** 🔥
- Вы можете **визуализировать, как ваш агент играет** 👀
- Вы можете **поделиться с сообществом агентом, которым могут пользоваться другие** 💾
- Вы можете **получить доступ к таблице лидеров 🏆, чтобы увидеть, насколько хорошо работает ваш агент по сравнению с вашими одноклассниками** 👉 https://huggingface.co/spaces/huggingface-projects/Deep-Reinforcement-Learning-Leaderboard

Чтобы иметь возможность поделиться своей моделью с сообществом, необходимо выполнить еще три шага:
1️⃣ (Если это еще не сделано) создайте учетную запись для HF ➡ https://huggingface.co/join

2️⃣ Войдите в систему, а затем вам нужно сохранить свой токен аутентификации с веб-сайта Hugging Face.
- Создайте новый токен (https://huggingface.co/settings/tokens) **с ролью записи**


<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/create-token.jpg" alt="Create HF Token" width="50%">


In [7]:
notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


Если вы не хотите использовать записную книжку Google Colabora Jupyter, вам нужно вместо этого использовать эту команду: `huggingface-cli login` (or `login`)

3️⃣ Теперь мы готовы отправить нашего обученного агента в " Хаб ", используя функцию `package_to_hub()`

In [38]:
repo_id = "Art-phys/Reinforce-1" #TODO Define your repo id {username/Reinforce-{model-id}}
push_to_hub(repo_id,
                cartpole_policy, # The model we want to save
                cartpole_hyperparameters, # Hyperparameters
                eval_env, # Evaluation environment
                video_fps=30
                )

/usr/local/lib/python3.8/dist-packages/gym/core.py:43: DeprecationWarning: WARN: The argument mode in render method is deprecated; use render_mode during environment initialization instead.
See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(


Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

model.pt:   0%|          | 0.00/3.92k [00:00<?, ?B/s]

Ваша модель помещена в центр. Вы можете просмотреть свою модель здесь: https://huggingface.co/Art-phys/Reinforce-1


Теперь, когда мы испытали надежность нашей реализации, давайте попробуем более сложную среду: PixelCopter 🚁


## Второй агент: PixelCopter 🚁

### Изучите среду PixelCopter 👀
- [Документация по окружающей среде](https://pygame-learning-environment.readthedocs.io/en/latest/user/games/pixelcopter.html)

In [8]:
env_id = "Pixelcopter-PLE-v0"
env = gym.make(env_id)
eval_env = gym.make(env_id)
s_size = env.observation_space.shape[0]
a_size = env.action_space.n

couldn't import doomish
Couldn't import doom


/usr/local/lib/python3.8/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.8/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [9]:
print("_____OBSERVATION SPACE_____ \n")
print("The State Space is: ", s_size)
print("Sample observation", env.observation_space.sample()) # Get a random observation

_____OBSERVATION SPACE_____ 

The State Space is:  7
Sample observation [ 0.04272867  1.6092854   0.10544054  0.3199548   1.1200747  -1.7607439
 -0.63699746]


In [10]:
print("\n _____ACTION SPACE_____ \n")
print("The Action Space is: ", a_size)
print("Action Space Sample", env.action_space.sample()) # Take a random action


 _____ACTION SPACE_____ 

The Action Space is:  2
Action Space Sample 1


Пространство наблюдений (7) 👀 :

- y позиция игрока
- скорость игрока
- расстояние игрока до пола
- расстояние игрока до потолка
- следующий блок x расстояние до игрока
- следующий блок местоположение остановки
- следующие блоки  y лучшее расположение

Пространство действий (2) 🎮 :
- Вверх
- вниз

Функция вознаграждения 💰 :

- За каждый вертикальный блок, через который он проходит, он получает положительную награду в размере +1. Каждый раз, когда достигается конечное состояние, оно получает отрицательное вознаграждение в размере -1.

### Определите новую политику 🧠
- Нам нужна более глубокая нейронная сеть, поскольку окружающая среда более сложная

In [11]:
class Policy(nn.Module):
    def __init__(self, s_size, a_size, h_size):
        super(Policy, self).__init__()
        # Задача: Определите здесь три слоя

    def forward(self, x):
        # Задача: Определите процесс пересылки здесь
        return F.softmax(x, dim=1)
    
    def act(self, state):
        state = torch.from_numpy(state).float().unsqueeze(0).to(device)
        probs = self.forward(state).cpu()
        m = Categorical(probs)
        action = m.sample()
        return action.item(), m.log_prob(action)

#### Solution

In [17]:
class Policy(nn.Module):
    def __init__(self, s_size, a_size, h_size):
        super(Policy, self).__init__()
        self.fc1 = nn.Linear(s_size, h_size)
        self.fc2 = nn.Linear(h_size, h_size*2)
        self.fc3 = nn.Linear(h_size*2, a_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return F.softmax(x, dim=1)
    
    def act(self, state):
        state = torch.from_numpy(state).float().unsqueeze(0).to(device)
        probs = self.forward(state).cpu()
        m = Categorical(probs)
        action = m.sample()
        return action.item(), m.log_prob(action)

### Определите гиперпараметры ⚙️
- Эта среда более сложная.
- Особенно для скрытого размера нам нужно больше нейронов.

In [27]:
pixelcopter_hyperparameters = {
    "h_size": 64,
    "n_training_episodes": 50000,
    "n_evaluation_episodes": 10,
    "max_t": 10000,
    "gamma": 0.99,
    "lr": 1e-4,
    "env_id": env_id,
    "state_space": s_size,
    "action_space": a_size,
}

### Тренируй это
- Теперь мы готовы обучать нашего агента 🔥 .

In [28]:
# Создайте политику и разместите ее на устройстве
# torch.manual_seed(50)
pixelcopter_policy = Policy(pixelcopter_hyperparameters["state_space"], pixelcopter_hyperparameters["action_space"], pixelcopter_hyperparameters["h_size"]).to(device)
pixelcopter_optimizer = optim.Adam(pixelcopter_policy.parameters(), lr=pixelcopter_hyperparameters["lr"])

In [29]:
scores = reinforce(pixelcopter_policy,
                   pixelcopter_optimizer,
                   pixelcopter_hyperparameters["n_training_episodes"], 
                   pixelcopter_hyperparameters["max_t"],
                   pixelcopter_hyperparameters["gamma"], 
                   1000)

Episode 1000	Average Score: 3.51
Episode 2000	Average Score: 5.25
Episode 3000	Average Score: 5.70
Episode 4000	Average Score: 7.61
Episode 5000	Average Score: 11.81
Episode 6000	Average Score: 12.17
Episode 7000	Average Score: 12.12
Episode 8000	Average Score: 16.16
Episode 9000	Average Score: 15.44
Episode 10000	Average Score: 12.32
Episode 11000	Average Score: 15.27
Episode 12000	Average Score: 15.91
Episode 13000	Average Score: 17.29
Episode 14000	Average Score: 17.20
Episode 15000	Average Score: 26.71
Episode 16000	Average Score: 20.48
Episode 17000	Average Score: 21.59
Episode 18000	Average Score: 25.47
Episode 19000	Average Score: 24.82
Episode 20000	Average Score: 25.73
Episode 21000	Average Score: 26.96
Episode 22000	Average Score: 28.32
Episode 23000	Average Score: 24.39
Episode 24000	Average Score: 31.26
Episode 25000	Average Score: 19.92
Episode 26000	Average Score: 38.31
Episode 27000	Average Score: 31.25
Episode 28000	Average Score: 32.39
Episode 29000	Average Score: 35.2

### Опубликуйте нашу обученную модель на хабе 🔥

In [31]:
repo_id = "Art-phys/Reinforce-Pixelcopter-PLE-v0" #Задача:Определите свой идентификатор репозитория {имя пользователя/Reinforce-{идентификатор модели}}
push_to_hub(repo_id,
                pixelcopter_policy, # Модель, которую мы хотим сохранить
                pixelcopter_hyperparameters, # Гиперпараметры
                eval_env, # Среда оценки
                video_fps=30
                )

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

model.pt:   0%|          | 0.00/38.9k [00:00<?, ?B/s]

Ваша модель помещена в центр. Вы можете просмотреть свою модель здесь: https://huggingface.co/Art-phys/Reinforce-Pixelcopter-PLE-v0


## Некоторые дополнительные проблемы 🏆
Лучший способ научиться ** - это пробовать что-то самостоятельно **! Как вы видели, у нынешнего агента дела идут не очень хорошо. В качестве первого предложения вы можете потренироваться для выполнения большего количества шагов. Но также пытаюсь найти лучшие параметры.

В [Таблице лидеров](https://huggingface.co/spaces/huggingface-projects/Deep-Reinforcement-Learning-Leaderboard) вы найдете своих агентов. Сможешь ли ты добраться до вершины?

Вот несколько идей для достижения этого:
* Тренируйте больше шагов
* Попробуйте разные гиперпараметры, посмотрев на то, что сделали ваши одноклассники 👉 https://huggingface.co/models?other=reinforce
* **Разместите свою новую обученную модель** на хабе 🔥
* **Улучшение реализации для более сложных сред ** (например, как насчет изменения сети на сверточную нейронную сеть для обработки
кадров в качестве наблюдения)?

** Поздравляю с завершением этого подразделения **! Там было много информации.
И поздравляю с окончанием урока. Вы только что закодировали свой первый обучающий агент с глубоким подкреплением с нуля с помощью PyTorch и поделились им на хабе 🥳.

Не стесняйтесь повторять этот модуль **, улучшая реализацию для более сложных сред** (например, как насчет изменения сети на сверточную нейронную сеть для обработки кадров в качестве наблюдения)?

В следующем разделе **мы собираемся узнать больше о Unity MLAgents**, обучая агентов в средах Unity. Таким образом, вы будете готовы принять участие в **AI vs AI challenges, где вы будете тренировать своих агентов
соревноваться с другими агентами в игре в снежки и футболе.**

Звучит забавно? Увидимся в следующий раз!

Увидимся в блоке 5! 🔥

### Продолжай учиться, оставайся потрясающим 🤗